In [7]:
import sqlite3
# подключаем базу данных
conn = sqlite3.connect('meeting_reminder.db')

# курсор для работы с таблицами
cursor = conn.cursor()

# try:
#     query = 'DROP TABLE "meeting_reminder"'
#     cursor.execute(query)
# except:
#     pass
#     raise

try:
    # sql запрос для создания таблицы
    query = 'CREATE TABLE "meeting_reminder" ("ID" INTEGER UNIQUE, "chat_id" INTEGER, "meet_time" TEXT, "meet_location" TEXT, "meet_name" TEXT, "meet_schedule" TEXT, "meet_text_msg" TEXT, PRIMARY KEY ("ID"))'
    cursor.execute(query)
except:
    pass
    raise

In [8]:
#--------------------------------------------------------------------    
def dop_meet_add(message):
    print('1------')
    try:
        meet_time = message.text.split(';')[0].strip()
        meet_location = message.text.split(';')[1].strip()
        meet_name = message.text.split(';')[2].strip()
        meet_schedule = message.text.split(';')[3].strip()
        
        #Доп преобразования
        meet_location = meet_location.replace('dion','').strip()
        meet_location = f'https://dion.vc/event/{meet_location}'
        print(meet_time, meet_location, meet_name,meet_schedule )
        
    except:
        text = 'Строка имеет неверный формат'
        bot.send_message(message.chat.id, text, reply_markup=keyboard_start)
        return None
    print('2------')
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        cursor.execute('INSERT INTO meeting_reminder (chat_id, meet_time, meet_location, meet_name, meet_schedule, meet_text_msg) VALUES (?, ?, ?, ?, ?, ?)',
                       (message.chat.id, meet_time, meet_location, meet_name, meet_schedule, message.text.strip()))
        con.commit()
    print('3------')
    text = 'Встреча добавлена в "календарь"'
    bot.send_message(message.chat.id, text, reply_markup=keyboard_start)

#--------------------------------------------------------------------
# просто функция, которая делает нам красивые строки для отправки пользователю
def get_plans_string(tasks):
    print('6------')
    tasks_str = []
    print(tasks)
    for val in list(enumerate(tasks)): 
        print(f'val = {val}')
        print(val[0])
        print(val[1][0])
        tasks_str.append(f'{str(val[0] + 1)}) {val[1][0]} {val[1][1]} {val[1][2]} {val[1][3]} \n')
    return ''.join(tasks_str)    

def dop_meet_show(message):
    print('5------')
    with sqlite3.connect('meeting_reminder.db') as con:
        cursor = con.cursor()
        cursor.execute('SELECT meet_time, meet_location, meet_name, meet_schedule, meet_text_msg FROM meeting_reminder WHERE chat_id=={}'.format(message.chat.id))
        tasks = get_plans_string(cursor.fetchall())
    print('7------')    
    bot.send_message(message.chat.id, tasks, reply_markup=keyboard_start)

In [9]:
import sqlite3
import telebot
from telebot import types
import json
import random
from datetime import datetime
from time import sleep


bot = telebot.TeleBot("6636328656:AAG5fr9YigHdxISXs8v7MqzYkeRF7tcysnI") # нужен токен вашего бота

# def handler(event,context): # основная функция для обработки действий бота
#     body = json.loads(event['body'])
#     update = telebot.types.Update.de_json(body)
#     bot.process_new_updates([update])

# делаем кнопки
button_start = types.KeyboardButton('Хочу смотреть мемы про программистов')
button_more = types.KeyboardButton('Неси еще')
button_stop = types.KeyboardButton('Хватит, пойду кодить')

button_meet_add = types.KeyboardButton('Добавить встречу')
button_meet_show = types.KeyboardButton('Показать встречи на сегодня')
button_meet_modify = types.KeyboardButton("Отредактировать встречу")
button_meet_delete = types.KeyboardButton('Удалить встречу')

# делаем две клавиатуры
keyboard_start = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
keyboard_start.add(button_meet_add, button_meet_show)
keyboard_start.add(button_meet_modify, button_meet_delete)

# keyboard_meet_delete = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
# keyboard_continue.add(button_more, button_stop)

# обрабатываем /start
@bot.message_handler(commands=['start'])
def start(message):
    start_text = "Привет! Я просто планирую встречи"
    bot.send_message(message.chat.id, start_text, reply_markup=keyboard_start)

# шедулер для отправки сообщений в чат
# @bot.message_handler(func = lambda message: True)
# def send_remind_message(message):
#     time_meet = datetime.strptime('1900-01-01 22:19', '%Y-%m-%d %H:%M').replace(year=1900, month=1, day=1, microsecond=0)
        
#     while False:
#         time_diff = (time_meet - datetime.now().replace(year=1900, month=1, day=1, microsecond=0)).seconds
# #         print(type(time_diff))
#         if time_diff >=0 and time_diff <= 120:
#             text = f'Встреча начнется через: {str(time_diff)} секунд'
#             break
#         elif time_diff < 0:
#             text = f'Встреча уже началась: {str(-time_diff)} секунд'
#             break
            
#         sleep(10)
#     text = 'ha'    
#     bot.send_message(message.chat.id, text, reply_markup=keyboard_start)
    
# добавление встречи
@bot.message_handler(regexp="Добавить встречу")
def meet_add(message):
    print('0------')
    bot.send_message(message.chat.id, 'Давай добавим встречу! Напиши её параметры в чат в формате: (ответить нужно на сообщение ниже)')
    msg = bot.send_message(message.chat.id, '9:30;dion skonkin;Daily;daily on weekdays')
    bot.register_next_step_handler(msg, dop_meet_add)

# показать все встречи]
@bot.message_handler(regexp="Показать встречи на сегодня")
def meet_show(message):
    print('4------')
    try:
        dop_meet_show(message)
    except:
        text = 'Здесь пусто. Не запланировано ниодной встречи'
        bot.send_message(message.chat.id, text, reply_markup=keyboard_start)
        raise # пока оставила для отладки в случае чего

    msg = bot.send_message(message.chat.id, 'Вот все встречи на сегодня')
    bot.register_next_step_handler(msg, dop_meet_show)

    
# обрабатываем другие сообщения
@bot.message_handler(content_types=["text"])
def repeat_all_messages(message):
    bot.send_message(message.chat.id, "Я тебя не понимаю" , reply_markup=keyboard_start)
    


    

In [10]:
bot.polling(none_stop=True)

0------
1------
15:20 https://dion.vc/event/skonkin Daily daily on weekdays
2------
3------
0------
1------
15:21 https://dion.vc/event/skonkin Daily daily on weekdays
2------
3------
0------
1------
15:22 https://dion.vc/event/skonkin Daily daily on weekdays
2------
3------
0------
1------
15:23 https://dion.vc/event/skonkin Daily daily on weekdays
2------
3------
4------
5------
6------
[('15:20', 'https://dion.vc/event/skonkin', 'Daily', 'daily on weekdays', '15:20;dion skonkin;Daily;daily on weekdays'), ('15:21', 'https://dion.vc/event/skonkin', 'Daily', 'daily on weekdays', '15:21;dion skonkin;Daily;daily on weekdays'), ('15:22', 'https://dion.vc/event/skonkin', 'Daily', 'daily on weekdays', '15:22;dion skonkin;Daily;daily on weekdays'), ('15:23', 'https://dion.vc/event/skonkin', 'Daily', 'daily on weekdays', '15:23;dion skonkin;Daily;daily on weekdays')]
val = (0, ('15:20', 'https://dion.vc/event/skonkin', 'Daily', 'daily on weekdays', '15:20;dion skonkin;Daily;daily on weekdays'

ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)

In [11]:
import sqlite3
with sqlite3.connect('meeting_reminder.db') as con:
    cursor = con.cursor()
    cursor.execute('SELECT * FROM meeting_reminder ')
    t = cursor.fetchall()
    
t

[(1, -4073737804, '2:30', 'dion pupu', 'tutututu', '2:30;dion pupu;tutututu'),
 (2, -4073737804, '4:30', 'dion pupu', 'tutututu', '4:30;dion pupu;tutututu'),
 (3, 365591342, '3:30', 'zxzx', 'cvcv', '3:30;zxzx;cvcv'),
 (4, 365591342, '6:30', 'zxzx', 'dion', '6:30;zxzx;dion'),
 (5, 365591342, '6:45', 'zxzx', 'dion', '6:45;zxzx;dion'),
 (6,
  -4073737804,
  '2:45',
  'dion pupu',
  'tutututu',
  '2:45; dion pupu; tutututu')]